# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

## This is a version of the reworked neural re-ranking model.
Main reason was that a dependancy hell happened on the Jupyter notebook.

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition..

In [4]:
!pip install --upgrade pip
!pip install numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.5 MB/s eta 0:00:00 0:00:01


In [5]:
import numpy as np
import pandas as pd

In [8]:
!pwd

/app


## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [9]:
PATH_COLLECTION_DATA = '/app/neural-re-ranking-reworked/subtask4b_collection_data.pkl' #MODIFY PATH

In [10]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [11]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [12]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [13]:
PATH_QUERY_TRAIN_DATA = '/app/neural-re-ranking-reworked/subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = '/app/neural-re-ranking-reworked/subtask4b_query_tweets_dev.tsv' #MODIFY PATH
PATH_QUERY_TEST_DATA = '/app/neural-re-ranking-reworked/subtask4b_query_tweets_test.tsv'

In [14]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')
df_query_test = pd.read_csv(PATH_QUERY_TEST_DATA, sep = '\t')

In [15]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [16]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [17]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


In [18]:
df_query_test.head()

,post_id,tweet_text
0,1,A recent research study published yesterday cl...
1,2,"""We should track the long-term effects of thes..."
2,3,"the agony of ""long haul"" covid-19 symptoms."
3,4,Home and online monitoring and assessment of b...
4,5,"it may be a long one, folks! to avoid exceedin..."


# 2) Running the baseline
The following code runs a BM25 baseline.


In [115]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [116]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [117]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:5]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [118]:
# Retrieve topk candidates using the BM25 model
df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))

# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [119]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [120]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

Results on the train set: {1: 0.5079747918773827, 5: 0.5508999196037242, 10: 0.5508999196037242}
Results on the dev set: {1: 0.505, 5: 0.5520357142857142, 10: 0.5520357142857142}


# 4) Exporting results to prepare the submission on Codalab

In [ ]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [ ]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')

## Initial Bi-Encoder Ranking with SentenceTransformer

In [20]:
!pip install -U sentence-transformers
!pip install keras==2.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 13.5 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 16.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 16.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 18.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 15.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 18.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 17.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 18.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━

In [21]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [22]:
# Extract text and identifiers
tweet_ids = df_query_test['post_id'].tolist()
tweet_texts = df_query_test['tweet_text'].tolist()
paper_ids = df_collection['cord_uid'].tolist()
paper_texts = df_collection['abstract'].tolist()

In [23]:
# Load fast BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
# Compute embeddings
tweet_embeds = model.encode(tweet_texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True)
paper_embeds = model.encode(paper_texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Batches:   0%|          | 0/121 [00:00<?, ?it/s]

In [27]:
# Compute cosine similarity between each tweet and all papers
similarity_matrix = cosine_similarity(tweet_embeds, paper_embeds)

In [28]:
# Get top 5 paper IDs for each tweet
top_k = 5
top5_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]
top5_cord_uids = [[paper_ids[i] for i in row] for row in top5_indices]
top5_cord_uids_str = ['[' + ', '.join(f'"{uid}"' for uid in ids) + ']' for ids in top5_cord_uids]

In [29]:
# Prepare submission DataFrame
output_df = pd.DataFrame({
    'post_id': tweet_ids,
    'preds': top5_cord_uids_str
})

# Save to TSV
output_df.to_csv('subtask4b_predictions.tsv', sep='\t', index=False)


In [30]:
output_df.head()

,post_id,preds
0,1,"[""8zufbeuz"", ""5aev7ltr"", ""x4zuv4jo"", ""26kx1b0h..."
1,2,"[""582ts5mf"", ""y8puocsa"", ""pofysmv8"", ""vw9jd88a..."
2,3,"[""gtp5daep"", ""25aj8rj5"", ""dikrttn6"", ""kb3xi0fk..."
3,4,"[""ru2ty1y9"", ""enlj85zc"", ""z43p1dmf"", ""vj5ahoi2..."
4,5,"[""0ul6k6uw"", ""lbrpbhpg"", ""3akrube4"", ""bfm45zas..."


In [34]:
import pandas as pd
import ast

preds_df = pd.read_csv('/app/neural-re-ranking-reworked/subtask4b_predictions.tsv', sep='\t')
train_df = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep='\t')

# Now this will succeed
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)

merged_df = train_df.merge(preds_df, on='post_id')

def get_performance_mrr(data, col_gold, col_pred, list_k=[1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(
            lambda x: (1 / ([i for i in x[col_pred][:k]].index(x[col_gold]) + 1)
                       if x[col_gold] in [i for i in x[col_pred][:k]] else 0),
            axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

results = get_performance_mrr(merged_df, col_gold='cord_uid', col_pred='preds')
print("MRR@k results:", results)


MRR@k results: {1: 0.0, 5: 0.000501930501930502, 10: 0.000501930501930502}


## BERT-based Pairwise Scoring and Reranking

In [61]:
# !pip3 install transformers
# !pip3 install tensorflow --upgrade --no-cache-dir
# !pip3 install tensorflow-gpu
!pip3 install transformers==4.46.3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [62]:
import tensorflow as tf
print(tf.__version__)

2.13.1


In [71]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import tensorflow as tf

# # Load tokenizer and model with classification head (1 output)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

import torch
from transformers import BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [75]:
def score_pairs(tweet, candidate_cord_uids, df_collection):
    inputs = []
    for uid in candidate_cord_uids:
        abstract = df_collection[df_collection['cord_uid'] == uid]['abstract'].values[0]
        inputs.append((tweet, abstract))

    # Tokenize all pairs
    encodings = tokenizer.batch_encode_plus(
    inputs,
    padding=True,
    truncation=True,
    return_tensors='pt'  # ✅ This returns PyTorch tensors
)

    # Predict relevance scores
    # outputs = model(encodings).logits  # Shape: (batch_size, 1)
    outputs = model(**encodings).logits
    # scores = tf.squeeze(outputs, axis=1).numpy()
    scores = outputs.squeeze(dim=1).detach().cpu().numpy()

    return list(zip(candidate_cord_uids, scores))

In [76]:
def rerank_topk(tweet_text, candidate_uids, df_collection):
    scored = score_pairs(tweet_text, candidate_uids, df_collection)
    reranked = sorted(scored, key=lambda x: x[1], reverse=True)
    return [uid for uid, score in reranked[:5]]

In [79]:
print(torch.cuda.is_available())

False


## Apply reranking on actual predictions data 
(As mentioned before, we have poor predictions here, this should be applied on better predictions)

In [77]:
from tqdm import tqdm
import ast

preds_df = pd.read_csv('/app/neural-re-ranking-reworked/subtask4b_predictions.tsv', sep='\t')
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)  # Convert to list

# Load test tweets
test_df = pd.read_csv(PATH_QUERY_TEST_DATA, sep='\t')

# Re-rank predictions using BERT
final_preds = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    post_id = row['post_id']
    tweet = row['tweet_text']

    # Get retrieved candidates
    candidate_uids = preds_df[preds_df['post_id'] == post_id]['preds'].values[0]

    # Run BERT-based re-ranking
    top5_reranked = rerank_topk(tweet, candidate_uids, df_collection)

    final_preds.append({
        'post_id': post_id,
        'preds': '[' + ', '.join(f'"{uid}"' for uid in top5_reranked) + ']'
    })


  0%|          | 0/1446 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 3/1446 [00:03<26:14,  1.09s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware

In [80]:
output_df = pd.DataFrame(final_preds)
output_df.to_csv('subtask4b_reranked_predictions.tsv', sep='\t', index=False)

In [81]:
import pandas as pd
import ast

# Load gold labels from train set
train_df = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep='\t')

# Load reranked predictions
preds_df = pd.read_csv('/app/neural-re-ranking-reworked/subtask4b_reranked_predictions.tsv', sep='\t')

# Convert stringified list to actual list
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)

# Merge predictions with gold labels (on post_id)
merged_df = df_query_train.merge(preds_df, on='post_id')


In [82]:
# Evaluation function (from notebook)
def get_performance_mrr(data, col_gold, col_pred, list_k=[1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(
            lambda x: (1 / ([i for i in x[col_pred][:k]].index(x[col_gold]) + 1)
                       if x[col_gold] in x[col_pred][:k] else 0),
            axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

# Run evaluation
results = get_performance_mrr(merged_df, col_gold='cord_uid', col_pred='preds')
print("MRR@k results:", results)


MRR@k results: {1: 0.0007722007722007722, 5: 0.001287001287001287, 10: 0.001287001287001287}


# Applying the predictions from the Traditional Model

In [88]:
TRADITIONAL_MODEL_PREDICTIONS = '/app/neural-re-ranking-reworked/traditional-model-predictions.tsv'

In [90]:
df_trad_model = pd.read_csv(TRADITIONAL_MODEL_PREDICTIONS, sep = '\t')

In [91]:
df_trad_model.head()

,post_id,preds
0,16,"['mb18fj8a', 'atji1xge', '66g5lpm6', 'gatxuwz7..."
1,69,"['r58aohnu', 'p0kg6dyz', 'j1ucrhd7', '9dlaaye8..."
2,73,"['gruir7aw', 'cgc0v1dg', 'sgo76prc', 'yi2m5ukw..."
3,93,"['3sr2exq9', 'sv48gjkk', 'k0f4cwig', 'z795y51f..."
4,96,"['3l6ipiwk', 'vabb2f26', 'ouvq2wpq', 'lzddnb8j..."


In [92]:
from tqdm import tqdm
import ast

preds_df = pd.read_csv(TRADITIONAL_MODEL_PREDICTIONS, sep='\t')
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)  # Convert to list

# Make sure post_id is the same type in both DataFrames
preds_df['post_id'] = preds_df['post_id'].astype(str)
test_df = pd.read_csv(PATH_QUERY_TEST_DATA, sep='\t')
test_df['post_id'] = test_df['post_id'].astype(str)

# Re-rank predictions using BERT
final_preds = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    post_id = row['post_id']
    tweet = row['tweet_text']

    # Get retrieved candidates
    filtered = preds_df[preds_df['post_id'] == post_id]
    if filtered.empty:
        print(f"Warning: No predictions for post_id {post_id}")
        continue

    candidate_uids = filtered['preds'].values[0]

    # Run BERT-based re-ranking
    top5_reranked = rerank_topk(tweet, candidate_uids, df_collection)

    final_preds.append({
        'post_id': post_id,
        'preds': '[' + ', '.join(f'"{uid}"' for uid in top5_reranked) + ']'
    })

  0%|          | 0/1446 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


  1%|          | 16/1446 [00:01<01:46, 13.44it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


  5%|▍         | 69/1446 [00:02<00:45, 30.39it/s]

  5%|▌         | 73/1446 [00:03<01:08, 20.03it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


  6%|▋         | 93/1446 [00:04<01:12, 18.78it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

 97%|█████████▋| 1398/1446 [02:53<00:05,  8.59it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 97%|█████████▋| 1402/1446 [02:54<00:06,  6.95it/s]

 98%|█████████▊| 1410/1446 [02:57<00:07,  4.93it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 99%|█████████▉| 1428/1446 [02:58<00:02,  7.80it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


100%|██████████| 1446/1446 [02:59<00:00,  8.05it/s]

In [96]:
output_df = pd.DataFrame(final_preds)
output_df.to_csv('/app/neural-re-ranking-reworked/subtask4b_reranked_predictions_from_basic_model.tsv', sep='\t', index=False)

In [113]:
import pandas as pd
import ast

# Load gold labels from train set
train_df = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep='\t')

# Load reranked predictions
preds_df = pd.read_csv('/app/neural-re-ranking-reworked/subtask4b_reranked_predictions_from_basic_model.tsv', sep='\t')

# Convert stringified list to actual list
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)

# Merge predictions with gold labels (on post_id)
merged_df = df_query_train.merge(preds_df, on='post_id')


In [114]:
# Run evaluation
results = get_performance_mrr(merged_df, col_gold='cord_uid', col_pred='preds')
print("MRR@k results:", results)


ValueError: Cannot set a DataFrame with multiple columns to the single column in_topx

# Applying the predictions from the NLP Model

In [101]:
NLP_MODEL_PREDICTIONS = '/app/neural-re-ranking-reworked/nlp-predictions.tsv'

In [102]:
df_nlp_model = pd.read_csv(TRADITIONAL_MODEL_PREDICTIONS, sep = '\t')

In [103]:
df_trad_model.head()

,post_id,preds
0,16,"['mb18fj8a', 'atji1xge', '66g5lpm6', 'gatxuwz7..."
1,69,"['r58aohnu', 'p0kg6dyz', 'j1ucrhd7', '9dlaaye8..."
2,73,"['gruir7aw', 'cgc0v1dg', 'sgo76prc', 'yi2m5ukw..."
3,93,"['3sr2exq9', 'sv48gjkk', 'k0f4cwig', 'z795y51f..."
4,96,"['3l6ipiwk', 'vabb2f26', 'ouvq2wpq', 'lzddnb8j..."


In [104]:
from tqdm import tqdm
import ast

preds_df = pd.read_csv(NLP_MODEL_PREDICTIONS, sep='\t')
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)  # Convert to list

# Make sure post_id is the same type in both DataFrames
preds_df['post_id'] = preds_df['post_id'].astype(str)
test_df = pd.read_csv(PATH_QUERY_TEST_DATA, sep='\t')
test_df['post_id'] = test_df['post_id'].astype(str)

# Re-rank predictions using BERT
final_preds = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    post_id = row['post_id']
    tweet = row['tweet_text']

    # Get retrieved candidates
    filtered = preds_df[preds_df['post_id'] == post_id]
    if filtered.empty:
        print(f"Warning: No predictions for post_id {post_id}")
        continue

    candidate_uids = filtered['preds'].values[0]

    # Run BERT-based re-ranking
    top5_reranked = rerank_topk(tweet, candidate_uids, df_collection)

    final_preds.append({
        'post_id': post_id,
        'preds': '[' + ', '.join(f'"{uid}"' for uid in top5_reranked) + ']'
    })

  0%|          | 0/1446 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 3/1446 [00:03<26:06,  1.09s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware

In [107]:
output_df = pd.DataFrame(final_preds)
output_df.to_csv('/app/neural-re-ranking-reworked/subtask4b_reranked_predictions_from_nlp.tsv', sep='\t', index=False)

In [109]:
import pandas as pd
import ast

# Load gold labels from train set
train_df = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep='\t')

# Load reranked predictions
preds_df = pd.read_csv('/app/neural-re-ranking-reworked/subtask4b_reranked_predictions_from_nlp.tsv', sep='\t')

# Convert stringified list to actual list
preds_df['preds'] = preds_df['preds'].apply(ast.literal_eval)

# Merge predictions with gold labels (on post_id)
merged_df = df_query_train.merge(preds_df, on='post_id')


In [110]:
# Run evaluation
results = get_performance_mrr(merged_df, col_gold='cord_uid', col_pred='preds')
print("MRR@k results:", results)

MRR@k results: {1: 0.0, 5: 0.0008365508365508364, 10: 0.0008365508365508364}
